# Imports

In [1]:
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

2023-05-13 10:56:43.642531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Extraction

In [33]:
image_dir = "train/"

INPUT_SIZE = 64

yes_pneumonia_images = os.listdir(image_dir+'PNEUMONIA')
no_pneumonia_images = os.listdir(image_dir+'NORMAL')
dataset = []
label = []


for i , image_name in enumerate(yes_pneumonia_images):
    if(image_name.split('.')[1] == 'jpeg'):
        image = cv2.imread(image_dir + 'PNEUMONIA/' + image_name)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image,'RGB')
        image = image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i , image_name in enumerate(no_pneumonia_images):
    if(image_name.split('.')[1] == 'jpeg'):
        image = cv2.imread(image_dir + 'NORMAL/' + image_name)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image,'RGB')
        image = image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

# Convert to numpy array and reshape

In [35]:
dataset = np.array(dataset)
label = np.array(label)

x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=.2, random_state=0)

# Normalize data and make models

In [36]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)




model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))



model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, 
          validation_data=(x_test,y_test), shuffle=False)

model.save('Pneumonia10Epochs.h5')

2023-05-13 11:36:34.148974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
261/261 [==============================] - 11s 35ms/step - loss: 0.0079 - accuracy: 0.9988 - val_loss: 9.6702e-27 - val_accuracy: 1.0000
Epoch 2/10
261/261 [==============================] - 9s 36ms/step - loss: 1.6578e-06 - accuracy: 1.0000 - val_loss: 5.3410e-29 - val_accuracy: 1.0000
Epoch 3/10
261/261 [==============================] - 10s 37ms/step - loss: 6.2522e-11 - accuracy: 1.0000 - val_loss: 5.3307e-29 - val_accuracy: 1.0000
Epoch 4/10
261/261 [==============================] - 9s 36ms/step - loss: 1.2012e-08 - accuracy: 1.0000 - val_loss: 3.5044e-29 - val_accuracy: 1.0000
Epoch 5/10
261/261 [==============================] - 9s 36ms/step - loss: 9.1630e-09 - accuracy: 1.0000 - val_loss: 2.4013e-29 - val_accuracy: 1.0000
Epoch 6/10
261/261 [==============================] - 10s 38ms/step - loss: 1.6632e-07 - accuracy: 1.0000 - val_loss: 4.1056e-32 - val_accuracy: 1.0000
Epoch 7/10
261/261 [==============================] - 10s 38ms/step - loss: 6.1622e-09 - accura

# Testing

In [46]:
model = load_model('Pneumonia10Epochs.h5')

#image = cv2.imread('/Users/siddhantborkar/pneumonia/test/NORMAL/IM-0001-0001.jpeg')
image = cv2.imread('/Users/siddhantborkar/pneumonia/test/PNEUMONIA/person83_bacteria_414.jpeg')
img = Image.fromarray(image)
img = img.resize((64,64))
img = np.array(img)


input_img = np.expand_dims(img, axis=0)
result = model.predict(input_img)
print(result)

1/1 [==============================] - 0s 69ms/step
[[0.]]
